<h1>Chapter 09. Decorators and Closures.</h1>

A Decorator is a callable object that accepts another function as an argument (the function to be decorated).
A Decorator can perform some operations on a function and returns either the function itself or another substitute function or called object.
Decorators are a powerful feature that allows you to modify or extend the behavior of functions or methods without directly modifying their code. They are typically used to add functionality such as logging, caching, or authentication to functions in a modular and reusable way. 

In [1]:
def deco(func):    
    def inner():
        print('Running inner() function')
        
    return inner  # deco() returns its inner() function object


@deco
def target():  # function decorated deco()
    ptint('Running target() function')


# Calling the decorated target() function actually executes inner()
target()

Running inner() function


Inspection shows that `target()` now refers to `inner()`

In [2]:
target

<function __main__.deco.<locals>.inner()>

<h2>When Python executes Decorators</h2>

The main feature of decorators is that they are perfomed as soon as the decorated function is defined. 

In [3]:
# Store references to functions decorated with @register
registry = []


def register(func):
    print(f"Running register({func})")  # print which function is being decorated
    registry.append(func)  # add func to registry
    return func


@register
def f1():
    print('Running f1()')


@register
def f2():
    print('Running f2()')


def f3():
    print('Running f3()')


print(f"Registry -> {registry}")
f1()
f2()
f3()

Running register(<function f1 at 0x112425ee0>)
Running register(<function f2 at 0x1125627a0>)
Registry -> [<function f1 at 0x112425ee0>, <function f2 at 0x1125627a0>]
Running f1()
Running f2()
Running f3()


<h2>Closures</h2>

A closure is a function, e.g. 'f', with an extended scope that covers variables referenced in the body of 'f', but which are neither global nor local variables of 'f'. Such variables must come from the local scope of the external function enclosing 'f'. Such variables must come from the local scope of the external function enclosing 'f'.

Class-based realization

In [4]:
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return f"The average value of {self.series} is {total / len(self.series)}"

In [5]:
avg = Averager()

avg(10)

'The average value of [10] is 10.0'

In [6]:
avg(11)

'The average value of [10, 11] is 10.5'

In [7]:
avg(12)

'The average value of [10, 11, 12] is 11.0'

Realization using the higher-order function `make_averager`

In [8]:
def make_averager():
    series = []

    # averager closure extends the scope of the functionto include
    # the binding of free variable series
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return f"The average value of {series} is {total / len(series)}"

    return averager

In [9]:
avg = make_averager()

avg(10)

'The average value of [10] is 10.0'

In [10]:
avg(11)

'The average value of [10, 11] is 10.5'

In [11]:
avg(12)

'The average value of [10, 11, 12] is 11.0'

Inspection of the function created by the `make_averager()` function

In [12]:
# Retrieve the variable names
avg.__code__.co_varnames

('new_value', 'total')

In [13]:
# Retrieve the free variable names
avg.__code__.co_freevars

('series',)

In [14]:
# Retrieve the closure
avg.__closure__

(<cell at 0x112557100: list object at 0x1124d48c0>,)

<h2><code>nonlocal</code> Announcement</h2>

`nonlocal` is a keyword in Python used within nested functions to indicate that a variable is not local to the innermost function's scope but is in an enclosing (non-global) scope. It allows you to modify variables in the enclosing scope from within the nested function.

Calculating the cumulative average without storing the entire history

In [15]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return f"The average value of {total} is {total / count}."

    return averager

In [16]:
avg = make_averager()

avg(10)

'The average value of 10 is 10.0.'

In [17]:
avg(11)

'The average value of 21 is 10.5.'

In [18]:
avg(12)

'The average value of 33 is 11.0.'